<a href="https://colab.research.google.com/github/BaoWhiteHat/-Code-solution-of-teacher-NTTMK/blob/main/LAB_3_EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Logo](https://www.uit.edu.vn/sites/vi/files/resize/images/Logos/Logo_UIT_Web_Transparent-218x261.png)

**Please clone this notebook into your own Google Drive. DO NOT EDIT or RUN CELL IN THIS NOTEBOOK!**

# **STUDENT INFORMATION**

**Full name**: Hoang-Loc Tran

**Student ID**: 13520462

**Class**:: CSBU108.O21

# **0. Write and Combine C program**

In [ ]:
%%writefile sample.cpp

#include <stdio.h>
#define NAME "LUCAS"

int main()
{
    printf("Hello world, %s!", NAME); // In dong chu "Hello world"

    return 0;
}

Overwriting sample.cpp


Compile file sample.cpp and then run it using shell command.

In [ ]:
!g++ sample.cpp -o sample
!./sample

Hello world, LUCAS!

**Example about command line arguments in C**

In [ ]:
%%writefile test_arg.cpp

#include <stdio.h>
#include <stdlib.h>

int main(int argc, char* argv[])
{
    if (argc < 2)
    {
        printf("We cannot find the name from command line!");
    }
    else
        printf("Hello world, %s %s\n", argv[1], argv[2]);
    return 0;
}

Overwriting test_arg.cpp


In [ ]:
!g++ test_arg.cpp -o test_arg
!./test_arg

We cannot find the name from command line!

In [ ]:
!./test_arg "Lucas Tran"

Hello world, Lucas Tran (null)


# **1. Process Creation**


## **1.1. Create process with `fork()`**

In [ ]:
%%writefile test_fork.cpp

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/types.h>

int main(int argc, char* argv[])
{
    __pid_t pid;
    pid = fork();

    if (pid > 0)
    {
        printf("PARENTS | PID = %ld | PPID = %ld\n",
        (long)getpid(), (long)getppid());

        if (argc > 2)
        {
            printf("PARENTS | There are %d arguments\n", argc - 1);
            wait(NULL);
        }
    }
    if (pid == 0)
    {
        printf("CHILDREN | PID = %ld | PPID = %ld\n",
        (long)getpid(), (long)getppid());

        printf("CHILDREN | List of arguments: \n");

        for (int i = 1; i < argc; i++)
        {
            printf("%s\n", argv[i]);
        }
    }

    return 0;
}

Writing test_fork.cpp


In [ ]:
!g++ test_fork.cpp -o test_fork
!./test_fork ThamSo1 ThamSo2 ThamSo3

PARENTS | PID = 15574 | PPID = 428
PARENTS | There are 3 arguments
CHILDREN | PID = 15575 | PPID = 15574
CHILDREN | List of arguments: 
ThamSo1
ThamSo2
ThamSo3


## **1.2. Create process with `exec()`**



**Firstly, create `count.sh` which do following tasks:**

*   Print process name
*   Print PPID (parent's process ID)
*   Count from 1 to $1 while printing numbers to `count.txt`

*Notes: the command `ps -ef | grep count.sh` lists all processes and then find which line (process' info) contains string "count.sh". This command helps us find information (PID and PPID) of process `./count.sh`*



In [ ]:
%%writefile count.sh

#!/bin/bash

echo "Implementing: $0"
echo "PPID of count.sh: "
ps -ef | grep count.sh

i=1

while [ $i -le $1 ]
do
    echo $i >> count.txt
    i=$((i + 1))
    sleep 1
done

exit 0

Writing count.sh


We have to grant execution permission to count.sh in order to execute it. We only do this once.

In [ ]:
!chmod 755 count.sh

Run the script (with $1 = 20) and check result if count.txt is created/updated.

In [ ]:
!./count.sh 20

Implementing: ./count.sh
PPID of count.sh: 
root       19761     428  0 03:56 ?        00:00:00 /bin/bash -c ./count.sh 20
root       19763   19761  0 03:56 ?        00:00:00 grep count.sh


**Create `test_execl.cpp`, compile and run it**

In [ ]:
%%writefile test_execl.cpp

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/types.h>

int main(int argc, char* argv[])
{
    __pid_t pid;
    pid = fork();
    if (pid > 0)
    {
        printf("PARENTS | PID = %ld | PPID = %ld\n",
        (long)getpid(), (long)getppid());

        if (argc > 2)
            printf("PARENTS | There are %d arguments\n",
            argc - 1);
        wait(NULL);
    }
    if (pid == 0)
    {
        execlp("./count.sh", "./count.sh", "10", NULL);

        printf("CHILDREN | PID = %ld | PPID = %ld\n",
        (long)getpid(), (long)getppid());

        printf("CHILDREN | List of arguments: \n");

        for (int i = 1; i < argc; i++)
        {
            printf("%s\n", argv[i]);
        }
    }

    return 0;
}


Writing test_execl.cpp


In [ ]:
!g++ test_execl.cpp -o test_execl
!./test_execl ThamSo1 ThamSo2

PARENTS | PID = 20284 | PPID = 428
PARENTS | There are 2 arguments
Implementing: ./count.sh
PPID of count.sh: 
root       20285   20284  0 03:58 ?        00:00:00 /bin/sh ./count.sh 10
root       20287   20285  0 03:58 ?        00:00:00 /bin/sh ./count.sh 10


## **1.3. Create process with `system()`**

**Create `test_system.cpp`, compile and run it**

In [ ]:
%%writefile test_system.cpp

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/types.h>

int main(int argc, char* argv[])
{
    printf("PARENTS | PID = %ld | PPID = %ld\n",
    (long)getpid(), (long)getppid());

    if (argc > 2)
        printf("PARENTS | There are %d arguments\n", argc
        - 1);

    system("./count.sh 10");

    printf("PARENTS | List of arguments: \n");

    for (int i = 1; i < argc; i++)
    {
        printf("%s\n", argv[i]);
    }

    return 0;
}

Overwriting test_system.cpp


In [ ]:
!g++ test_system.cpp -o test_system
!./test_system ThamSo1 ThamSo2 ThamSo3

PARENTS | PID = 26936 | PPID = 26787
PARENTS | There are 3 arguments
Implementing: ./count.sh
PPID of count.sh: 
root       26937   26936  0 04:26 ?        00:00:00 sh -c ./count.sh 10
root       26938   26937  0 04:26 ?        00:00:00 /bin/sh ./count.sh 10
root       26940   26938  0 04:26 ?        00:00:00 grep count.sh
PARENTS | List of arguments: 
ThamSo1
ThamSo2
ThamSo3


# **2. Inter-Process Communication: Message passing**


In [ ]:
%%writefile test_pipe.cpp

#include <stdio.h>
#include <unistd.h>
#include <string.h>
#include <sys/types.h>
#include <sys/wait.h>

int main() {
    int pipefd[2];
    pid_t pid;
    char write_msg[] = "Hello from parent";
    char read_msg[100];

    // Create the pipe
    if (pipe(pipefd) == -1) {
        perror("pipe");
        return 1;
    }

    // Fork a child process
    pid = fork();
    if (pid == -1) {
        perror("fork");
        return 1;
    }

    if (pid == 0) {
        // Child process
        close(pipefd[1]); // Close the write end

        // Read the message from the pipe
        read(pipefd[0], read_msg, sizeof(read_msg));
        printf("Child process received message: %s\n", read_msg);

        close(pipefd[0]); // Close the read end
    } else {
        // Parent process
        close(pipefd[0]); // Close the read end

        // Write the message to the pipe
        write(pipefd[1], write_msg, strlen(write_msg) + 1);

        close(pipefd[1]); // Close the write end

        // Wait for the child process to finish
        wait(NULL);
    }

    return 0;
}

Overwriting test_pipe.cpp


In [ ]:
!g++ test_pipe.cpp -o test_pipe
!./test_pipe

Child process received message: Hello from parent


# **ANSWERS**

## **Classwork 01**

## **Classwork 02**

## **Classwork 03**

## **Classwork 04**